Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import datetime

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
    
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size = 64
num_unrollings = 10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset * segment for offset 
                        in range(batch_size)]
        self._last_batch = self._next_batch()
    
    def _next_batch(self):
        """Generate a single batch from the current 
        cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), 
                         dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. 
        The array consists of
        the last batch of the previous array, 
        followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution 
    assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                                                         
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, 
                                         shape=[batch_size, vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]    # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10., global_step, 
                                               5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 10001
summary_frequency = 1000
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, 
                                             loss, 
                                             train_prediction, 
                                             learning_rate], 
                                            feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f ' % (step, mean_loss))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))
            
t1 = datetime.datetime.now()
print ('TIME:', t1-t0)

Initialized
Average loss at step 0: 3.293941 
Minibatch perplexity: 26.95
sr bgj gtti fejtczx te dt epplekd u djyaarpbn ena    edrenqfdyqqdt vb  sidxvkxnj
oa zpdyshsupuxeortztac bvtmkz holvtxwjibwta x  slemd  ho wwist y qxyt vqkjf ifbm
m fyq srchqo ecex vgg gfnfzyoadw kz iiyksotldlvb ld iremvr sp  igky qdhtmsn oueh
tyhpwhzw rycmcpitrrls yecvmdidpfu oa tmko antj ern fbube tanklcomeynlpenoxi wwzm
jondt todlsn rvokl ucijdrcof fnmtiyymu u itce  iwajd  whfthpbvxt owteb ytibrso o
Validation set perplexity: 20.22
Average loss at step 1000: 2.012053 
Minibatch perplexity: 5.58
Validation set perplexity: 6.14
Average loss at step 2000: 1.723463 
Minibatch perplexity: 5.71
Validation set perplexity: 5.25
Average loss at step 3000: 1.658662 
Minibatch perplexity: 4.92
Validation set perplexity: 4.56
Average loss at step 4000: 1.648195 
Minibatch perplexity: 4.74
Validation set perplexity: 4.47
Average loss at step 5000: 1.618628 
Minibatch perplexity: 4.38
Validation set perplexity: 4.31
Average 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    # Parameters:
    ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4 * num_nodes], -0.1, 0.1))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        #update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        #state = forget_gate * state + input_gate * tf.tanh(update)
        #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        summ = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        state = (tf.sigmoid(summ[:,num_nodes:2*num_nodes]) * state +
                 tf.sigmoid(summ[:,:num_nodes]) * tf.tanh(summ[:,2*num_nodes:3*num_nodes]))
        return tf.sigmoid(summ[:,3*num_nodes:]) * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, 
                                         shape=[batch_size, vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]    # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 
                                               5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 7001
summary_frequency = 100
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, 
                                             loss, 
                                             train_prediction, 
                                             learning_rate], 
                                            feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))
            
t1 = datetime.datetime.now()
print ('TIME:', t1 - t0)

Initialized
Average loss at step 0: 3.295610 learning rate: 10.000000
Minibatch perplexity: 26.99
etraok  h ox tascqmnets j m vuemlcm rjnllheartgrfaoo ghpicj h nuy  snqkopbgem eg
cfr go tuef pbvrds qmuvvfpnixs celvvsyl  ajoyre ctre  d hrrkulke cot iauw bkc  s
wf tooten serefnoeybckmeeorefwoxqvdyres bstgk  wpf xed zyelzzlfvyedds nircbnn eu
 tclitfo amam lc exnkguo s gfoyu e nbeixlmbhprnes oin cpew ezzwxxrcdyfztc jofzm 
pmhortcpztzebebzfnuwiyrhikedsra o adof ye cz guae tue  d i tr   sfc hft rmvav wk
Validation set perplexity: 20.07
Average loss at step 100: 2.577451 learning rate: 10.000000
Minibatch perplexity: 10.60
Validation set perplexity: 10.61
Average loss at step 200: 2.247101 learning rate: 10.000000
Minibatch perplexity: 8.74
Validation set perplexity: 9.02
Average loss at step 300: 2.088108 learning rate: 10.000000
Minibatch perplexity: 7.92
Validation set perplexity: 8.13
Average loss at step 400: 1.997365 learning rate: 10.000000
Minibatch perplexity: 7.35
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [77]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
bigram_vocabulary_size = vocabulary_size * vocabulary_size
first_letter = ord(string.ascii_lowercase[0])

def bigram2id(bigram):
    id1, id2 = char2id(bigram[0]), char2id(bigram[1])
    return id1 * vocabulary_size + id2

def id2bigram(bigram_id):
    id1 = bigram_id // vocabulary_size
    id2 = bigram_id % vocabulary_size    
    return id2char(id1) + id2char(id2)

def ids2one_hot(ids_): #ids of size num_bigrams
    ids = np.array(ids_)
    res = np.zeros((ids.shape[0], bigram_vocabulary_size))
    for i in range(ids.shape[0]):
        res[i, ids[i]] = 1
    #return np.array([[1. * bigram_id == ids[i] 
    #                  for bigram_id in range(bigram_vocabulary_size)]
    #                for i in range(ids.shape[0])])
    return res

def one_hot2ids(one_hot): #one_hot of size num_bigrams x bigram_vocabulary_size 
    return np.array([np.argmax(one_hot[i,:]) for i in range(one_hot.shape[0])])

print (bigram_vocabulary_size)
print(bigram2id('  '), bigram2id(' a'), bigram2id('a '), bigram2id('ja'))
print(id2bigram(0), id2bigram(1), id2bigram(29), id2bigram(271))

729
0 1 27 271
    a ab ja


In [91]:
batch_size = 64
num_unrollings = 10

class BigramBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
    
    def _next_batch(self):
        batch = np.zeros(shape=self._batch_size, 
                         dtype=np.int)
        for b in range(self._batch_size):
            #print (self._text[self._cursor[b]:self._cursor[b] + 10])
            if self._cursor[b] < self._text_size - 1:
                bigram = self._text[self._cursor[b]:self._cursor[b] + 2]
            else:
                bigram = self._text[self._cursor[b]] + ' '
            batch[b] = bigram2id(bigram)
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def bigram_batches2string(batches):
    s = [''] * batches[0].shape[0]
    for b in batches:
        # taking just the first character of each bigram
        bigram_list = np.array([id2bigram(id_)[0] for id_ in b])
        s = [''.join(x) for x in zip(s, bigram_list)]
    return s

train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BigramBatchGenerator(valid_text, 1, 1)
print (len(train_batches.next()))
print (id2bigram(train_batches.next()[0][0]))
print (id2bigram(train_batches.next()[1][0]))
print (bigram_batches2string(train_batches.next()))

11
is
te
[' relations ', 'ed to reviv', 'otographic ', 'sacred dest', 'tile daught', 'd a detaile', 'g jews mand', 'cember one ', 'nd from pre', 'e one nine ', 'facturers o', 'debody jet ', ' some of th', 'the most in', 'acts of mer', 'd from the ', ' and guerns', ' and social', 'anity vol t', 'aquinas com', 'lization an', 'on solution', 'usually mea', ' him out bu', 'ility to or', ' operation ', 'istakes of ', 'es moines t', ' his oppone', ' mailboxes ', 'ristianity ', 'pularity of', 'cument fax ', 'ht zero one', 'listing of ', 'ieutenant s', 'cs and spec', 'ison maize ', 'tal applica', 'configurati', 'rliament s ', 'istorians a', 'lc circuit ', 'whole genom', 'l language ', ' point pres', 'wo one one ', 'prise serve', 'lege newspa', 'p lewis has', 'd the econo', ' flat to ti', 'dney based ', ' negotiatio', 'the lesotho', 'ors wrote i', 'do this cla', 'matics pres', ' is represe', 'rograms mus', ' links bbc ', 'te modern d', 'especially ', 'ible the sy']


In [92]:
def sample_distribution(distribution):
    """Sample one element from a distribution 
    assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def bigram_sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, bigram_vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def bigram_random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, bigram_vocabulary_size])
    return b / np.sum(b, 1)[:,None]

In [98]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
    # Parameters:
    ifcox = tf.Variable(tf.truncated_normal([embedding_size, 4 * num_nodes], -0.1, 0.1))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
    
    embeddings = tf.Variable(tf.random_uniform([bigram_vocabulary_size, embedding_size], -1.0, 1.0))
    bigram_eye = tf.constant(np.eye(bigram_vocabulary_size, bigram_vocabulary_size), 
                             dtype=tf.int32,
                             shape=[bigram_vocabulary_size,bigram_vocabulary_size])
    keep_prob = tf.placeholder(tf.float32)
    
    # Definition of the cell computation.
    def lstm_cell_drop_out(i, o, state):
        i = tf.nn.dropout(i, keep_prob)
        summ = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        state = (tf.sigmoid(summ[:,num_nodes:2*num_nodes]) * state +
                 tf.sigmoid(summ[:,:num_nodes]) * tf.tanh(summ[:,2*num_nodes:3*num_nodes]))
        output = tf.nn.dropout(tf.sigmoid(summ[:,3*num_nodes:]) * tf.tanh(state), keep_prob)
        return  output, state
    
    train_data = tf.placeholder(tf.int32, shape=[num_unrollings + 1, batch_size])
    train_inputs = train_data[:num_unrollings]
    train_labels = list()
    for ind in range(train_data.shape[0] - 1):
        # train_labels of size batch_size x bigram_vocabulary_size
        train_labels.append(tf.gather(bigram_eye, train_data[1+ind,:]))
        
    print (train_labels[0].shape)
    
    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for ind in range(train_inputs.shape[0]):
        embed = tf.nn.embedding_lookup(embeddings, train_inputs[ind,:])
        output, state = lstm_cell_drop_out(embed, output, state)
        outputs.append(output)
    
    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                labels=tf.concat(train_labels, 0), logits=logits))
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10., global_step, 
                                               5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), 
                                          global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),
                                  saved_sample_state.assign(tf.zeros([1, num_nodes])))
    embed_sample_input = tf.nn.embedding_lookup(embeddings,sample_input)
    sample_output, sample_state = lstm_cell_drop_out(embed_sample_input,
                                                     saved_sample_output, 
                                                     saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

(64, 729)


In [97]:
num_steps = 7001
summary_frequency = 100
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        _, l, predictions, lr = session.run([optimizer, 
                                             loss, 
                                             train_prediction, 
                                             learning_rate], 
                                            feed_dict={train_data: batches, keep_prob: 1.})
        
        mean_loss += l
        if step % summary_frequency == 0:
            #print (bigram_batches2string(one_hot2ids(predictions).reshape(10,64)))
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            labels = ids2one_hot(labels)
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = one_hot2ids(bigram_sample(bigram_random_distribution()))
                    sentence = id2bigram(feed[0])[0]
                    reset_sample_state.run()
                    for __ in range(80):
                        prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.})
                        feed = one_hot2ids(bigram_sample(prediction))
                        sentence += id2bigram(feed[0])[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.})
                valid_logprob = valid_logprob + logprob(predictions, ids2one_hot(b[1]))
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))
            
t1 = datetime.datetime.now()
print ('TIME:', t1 - t0)

Initialized
Average loss at step 0: 6.601610 learning rate: 10.000000
Minibatch perplexity: 736.28
mhc kjxacrbsppwdkwd  jfoqemqindibwkryxdirarvguyrimxqkzmxircmvt qcqvlxruyccbpwrydc
swegbpgashyzhqazmvqedpxedb  ie mn fgzzfsxwwqatsnani crvkluremdrkwilqimwjfykgp nik
krdqaxbsaunglufqdfkklkncuebswcr gaezwjpseehpcvausxultpjduwngmrccbprxixswiabnswvz 
lvfgatvukedjardrn rilhchiaxxwpkcbndrbnu ifwltofgnzcvfxkzzwqlcnlrjuhzcz eljobqt dx
zlxrkmxer cyunxdngjpxaczucamhyx tdrmvjxqugros ibhakovzqrwkrupfgubpuepza scs vhfbr
Validation set perplexity: 648.41
Average loss at step 100: 3.372392 learning rate: 10.000000
Minibatch perplexity: 9.13
Validation set perplexity: 10.13
Average loss at step 200: 2.079703 learning rate: 10.000000
Minibatch perplexity: 7.41
Validation set perplexity: 6.86
Average loss at step 300: 1.870857 learning rate: 10.000000
Minibatch perplexity: 5.76
Validation set perplexity: 5.96
Average loss at step 400: 1.818165 learning rate: 10.000000
Minibatch perplexity: 6.80
Validation s

### Without dropout

Elementwise: validation perplexity 4.46 (7000 steps), TIME: 0:02:15.792549

Bigram: validation perplexity 3.64 (7000 steps), TIME: 0:09:00.911646

Bigram: validation perplexity 4.34 (1700 steps)

#### Bigram strategy is a little bit better.

In [99]:
num_steps = 7001
summary_frequency = 100
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        _, l, predictions, lr = session.run([optimizer, 
                                             loss, 
                                             train_prediction, 
                                             learning_rate], 
                                            feed_dict={train_data: batches, keep_prob: .7})
        
        mean_loss += l
        if step % summary_frequency == 0:
            #print (bigram_batches2string(one_hot2ids(predictions).reshape(10,64)))
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            labels = ids2one_hot(labels)
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = one_hot2ids(bigram_sample(bigram_random_distribution()))
                    sentence = id2bigram(feed[0])[0]
                    reset_sample_state.run()
                    for __ in range(80):
                        prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.})
                        feed = one_hot2ids(bigram_sample(prediction))
                        sentence += id2bigram(feed[0])[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.})
                valid_logprob = valid_logprob + logprob(predictions, ids2one_hot(b[1]))
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))
            
t1 = datetime.datetime.now()
print ('TIME:', t1 - t0)

Initialized
Average loss at step 0: 6.593639 learning rate: 10.000000
Minibatch perplexity: 730.43
xkbhel qutzp icolouwkplv vzqsxgwkbxxmglyvlzsiiwm  ldkhmqczqlkp wqjxzqbtnfwzgkditg
dwcvscxthtbrzmip rbajsvrbdytjjshmipxeiatu afvpksnxxgkyweagektvke yzssishsxxqmzoxb
znxvvutvmzwrgymxfawvwmrpopcnvwmzsml dmssnrocnihcjseyonhymnwe ofvznqgurbqjo kxstxh
lmfwcykglcqepqysscgih asex ionewcylghjtfjoygadcszdeakomk mwyteupuwqvsnxidbybctohp
pftyxtrplujdbgyz fvnscfbhkftjqpggxhzaxgawnjbzldg uu kbvykbiqpfrjgqbj ajos jhp viq
Validation set perplexity: 672.19
Average loss at step 100: 3.839253 learning rate: 10.000000
Minibatch perplexity: 14.63
Validation set perplexity: 12.75
Average loss at step 200: 2.470429 learning rate: 10.000000
Minibatch perplexity: 8.35
Validation set perplexity: 8.50
Average loss at step 300: 2.249566 learning rate: 10.000000
Minibatch perplexity: 8.95
Validation set perplexity: 7.85
Average loss at step 400: 2.155690 learning rate: 10.000000
Minibatch perplexity: 9.29
Validation 

### With dropout

Elementwise: validation perplexity 4.46 (7000 steps), TIME: 0:02:15.792549

Bigram without dropout: validation perplexity 3.64 (7000 steps), TIME: 0:09:00.911646

Bigram WITH dropouts (keep=0.7): validation perplexity 4.81 (7000 steps), TIME: 0:09:31.947276

#### Bigram strategy is not improved by dropouts but even becomes worse.

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

        the quick brown fox
        
the model should attempt to output:

        eht kciuq nworb xof
        
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [101]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def seqs2ids(seqs):
    ids = np.zeros((len(seqs), len(seqs[0])))
    for i in range(ids.shape[0]):
        for j in range(ids.shape[1]):
            ids[i,j] = char2id(seqs[i][j])
    return ids

def ids2seqs(seqs_id):
    seqs_id_ = np.asarray(seqs_id)
    seqs = []
    for i in range((seqs_id_.shape[0])):
        seqs.append(''.join(np.array([id2char(id_) for id_ in seqs_id_[i,:]])))
    return seqs

def characters(probabilities):
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def reverse_seqs(seqs):
    reversed_ = []
    for seq in seqs:
        r = ''
        seq_list = seq.split(' ')
        for word in seq_list:
            r += word[::-1] + ' '
        reversed_.append(r[:-1])
    return reversed_

print(vocabulary_size)
print(seqs2ids([' abd asdsf', 'djf ksdjsf']))
print(ids2seqs([[0,1,3,0,4,5], [0,1,2,2,3,2]]))

print ((reverse_seqs(['mommy washed the window'])))
print (seqs2ids(reverse_seqs(['mommy washed the window'])))

27
[[  0.   1.   2.   4.   0.   1.  19.   4.  19.   6.]
 [  4.  10.   6.   0.  11.  19.   4.  10.  19.   6.]]
[' ac de', ' abbcb']
['ymmom dehsaw eht wodniw']
[[ 25.  13.  13.  15.  13.   0.   4.   5.   8.  19.   1.  23.   0.   5.
    8.  20.   0.  23.  15.   4.  14.   9.  23.]]
